In [1]:
from AnthropicWrapper import ClaudeChatDocument

In [2]:
model = "claude-3-5-sonnet-20240620"
pdf_path = "D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"


In [3]:
str2 = "Machine Learning Engineer"

In [4]:
system_prompt = f"You are a skilled interviewer who is conducting a screening interview for a candidate for a {str2} role. Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the role, which may or may not be based on the interviewee's CV, which you have access to. Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by thanking the candidate for their time and informing them that they will receive word soon on the outcome of the screening interview. The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview."

In [5]:
print(system_prompt)

You are a skilled interviewer who is conducting a screening interview for a candidate for a Machine Learning Engineer role. Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the role, which may or may not be based on the interviewee's CV, which you have access to. Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by thanking the candidate for their time and informing them that they will receive word soon on the outcome of the screening interview. The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview.


In [6]:
claude = ClaudeChatDocument(model, system_prompt, pdf_path)

In [7]:
while True:
    print(claude.chat_with_history_doc(input()))

Hello! It's wonderful to meet you as well. Thank you for coming in today for this screening interview for the Machine Learning Engineer position. I'm excited to learn more about you and your experience. To start off, could you tell me a bit about what attracted you to this role and why you're interested in working as a Machine Learning Engineer?
I appreciate your enthusiasm! It's great that you find the role exciting. Could you elaborate a bit more on what specific aspects of machine learning engineering appeal to you? I see from your CV that you're completing a Master's in Artificial Intelligence and have worked on some interesting projects. What areas of machine learning are you particularly passionate about?
I appreciate your honesty. It's okay if you haven't pinpointed a specific area of focus yet. Machine learning is a broad field with many exciting applications. Given your educational background in AI and your project experience, perhaps we could approach this from a different an

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.16: all messages must have non-empty content except for the optional final assistant message'}}

In [8]:
testlist = []
for message in claude.get_message_history():
    if message.get("role") == "user" and "The following is a document that I am providing you with." in message.get("content", ""):
        testlist.append("Document Included, please use ClaudeChatDocument.get_document() to view document\n")
        testlist.append("-" * 20 + "\n") 
    else:
        for key, value in message.items():
            testlist.append(f"{key}: {value}\n")
        testlist.append("-" * 20 + "\n") 

yeet = "".join(testlist)

print(yeet)

Document Included, please use ClaudeChatDocument.get_document() to view document
--------------------
role: assistant
content: Understood! I'll keep this in the back of my mind.
--------------------
role: user
content: Hi nice to meet you!
--------------------
role: assistant
content: Hello! It's wonderful to meet you as well. Thank you for coming in today for this screening interview for the Machine Learning Engineer position. I'm excited to learn more about you and your experience. To start off, could you tell me a bit about what attracted you to this role and why you're interested in working as a Machine Learning Engineer?
--------------------
role: user
content: Yeah forsure! the role is pretty cool ngl
--------------------
role: assistant
content: I appreciate your enthusiasm! It's great that you find the role exciting. Could you elaborate a bit more on what specific aspects of machine learning engineering appeal to you? I see from your CV that you're completing a Master's in Arti

In [12]:
import re
from fpdf import FPDF

In [29]:
conversation = yeet

In [42]:
# Function to extract role and content from each turn
def parse_turn(turn):
    role_match = re.search(r"role: (.*)", turn)
    content_match = re.search(r"content: (.*)", turn, re.DOTALL)

    role = role_match.group(1) if role_match else ""
    content = content_match.group(1).strip() if content_match else turn.strip()

    return role, content

# Create a list of dictionaries, each representing a turn in the conversation
conversation_list = []
for turn in conversation.split("--------------------\n"):
    if turn.strip():  # Ignore empty turns
        role, content = parse_turn(turn)
        if content != "Understood! I'll keep this in the back of my mind.": 
            conversation_list.append({"role": role, "content": content})

# Create a PDF object
pdf = FPDF()
pdf.add_page()

# --- Improved Formatting ---

# Set margins (in millimeters)
pdf.set_margins(left=10, top=20, right=10)  # Right margin set to 10mm 

# Choose a nicer font
pdf.set_font("Helvetica", size=12)

# Calculate usable width for text wrapping (accounting for margins)
usable_width = pdf.w - pdf.l_margin - pdf.r_margin -0 # 10px right margin 

# Add conversation to the PDF
for turn in conversation_list:
    # Skip turns with empty roles or content
    if turn['role'] and turn['content']:
        # Subheading style for "User" and "Assistant"
        pdf.set_font("Helvetica", style="B", size=14)
        pdf.cell(0, 10, txt=f"{turn['role'].capitalize()}:", ln=True)

        # Content with regular font and correct indentation
        pdf.set_font("Helvetica", size=12)
        pdf.x = pdf.l_margin  # Reset x-coordinate to the left margin
        pdf.multi_cell(usable_width, 10, txt=turn['content'])
        pdf.ln()

# Save the PDF
pdf.output("conversation.pdf")

''